In [ ]:
mport datetime
import heliosat
import matplotlib
import numpy as np
import py3dcore
import pickle

from matplotlib import rcParams
rcParams["font.size"] = 16

from matplotlib import pyplot as plt
import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D

c0 = "xkcd:black"
c1 = "xkcd:magenta"
c2 = "xkcd:orange"
c3 = "xkcd:azure"

if __name__ == "__main__":
    t_l = datetime.datetime(2018, 11, 11, 6)

    iparams_arr = np.array([np.median(obj.particles, axis=0)], dtype=np.float32)
    #iparams_arr[0, 5] = 2
    
    t_0 = datetime.datetime(2018, 11, 12)
    t_s = [t_0 + datetime.timedelta(hours=i) for i in range(-14*24, 14*24)]

    
    # thin torus
    model_obj2 = py3dcore.models.ThinTorusGH3DCOREModel(t_l, runs=1, use_gpu=False)
    model_obj2.update_iparams(iparams_arr, seed=42)
    model_obj2.propagate(t_0)
    
    fig = plt.figure(figsize=(16, 16 ))
    ax = fig.gca(projection='3d')

    wf_model2 = model_obj2.visualize_wireframe(index=0)

    ax.plot_wireframe(wf_model2[:, :, 0], wf_model2[:, :, 1], wf_model2[:, :, 2], color="k", alpha=.1, lw=1.5)

    ax.set_axis_off()
    ax.view_init(elev=35, azim=240)
    ax.set_xlim([-.65, .65])
    ax.set_ylim([-.65, .65])
    ax.set_zlim([-.65, .65])
    
   
    
    # earth
    
    colors = ["g", "r", "k", "y"]
    objs = [heliosat.Earth(), heliosat.STA(), heliosat.PSP(), heliosat.Sun()]
    texts = ["Earth", "STEREO-A", "PSP", ""]
    offsett = [
        np.array([-.2, .2, 0]),
        np.array([.05, 0, 0]),
        np.array([-.25, 0, .05]),
        np.array([.1, 0, 0])
    ]
    
    i = 0
    for objs_ in objs:
        c = colors[i]

        position = objs_.trajectory(t_0, frame="HCI")
        path = objs_.trajectory(t_s, frame="HCI")
        ax.scatter(*position, color=c, s=500)
        ax.plot(*path.T, color=c, ls="-")
        tpos = position + offsett[i]
        ax.text(*tpos, texts[i], fontsize=32)

        
        i += 1
        
    r = np.linspace(0, 1.2, 5)
    phi_0 = -.38
    phi = np.linspace(phi_0, 2*np.pi+phi_0 , 36)
       
    dr, dphi = np.meshgrid(r, phi)
    
    dx = dr * np.cos(dphi)
    dy = dr * np.sin(dphi)
    dz = np.zeros_like(dr)

    ax.plot_wireframe(dx, dy, dz, color="k", ls="--", alpha=0.25)
    
    
    plt.savefig("out/40_fitting_model.pdf")